### Train a 2-layer LSTM network to find patterns in time-series data.

In [8]:
%matplotlib inline


import numpy as np
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
# from torch.autograd import Variable

In [ ]:
class Sequence(nn.Module):
    def __init__(self, hidden_size=50):
        super(Sequence, self).__init__()
        
        # Hidden layer.
        self.hidden_size = hidden_size + 1
        
        # Network architecture.
        self.lstm1 = nn.LSTMCell(input_size=1,
                                 hidden_size=self.hidden_size)
        self.lstm2 = nn.LSTMCell(input_size=self.hidden_size,
                                 hidden_size=self.hidden_size)
        self.linear = nn.Linear(in_features=self.hidden_size,
                                out_features=1)


    def forward(self, inputs:torch.tensor, future=0):
        # print(inputs)
        input_size = inputs.size()
        
        # Hidden & cell state of 1st LSTM layer.
        h_t1 = torch.zeros(input_size, self.hidden_size,
                           dtype=torch.double)
        c_t1 = torch.zeros(input_size, self.hidden_size,
                           dtype=torch.double)
        
        # Hidden & cell state for 2nd LSTM layer.
        h_t2 = torch.zeros(input_size, self.hidden_size,
                           dtype=torch.double)
        c_t2 = torch.zeros(input_size, self.hidden_size,
                           dtype=torch.double)
        
        # Outputs
        outputs = []
        for t, input_t in enumerate(inputs.chunk(input_size, dim=1)):
            h_t1, c_t1 = self.lstm1(input=input_t, weight=(h_t1, c_t1))
            h_t2, c_t2 = self.lstm2(h_t1, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs.append(output)
            
        # If we should predict the future.
        for i in range(future):
            pass